In [395]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
import math
cv2.startWindowThread()


0

In [396]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/1080_120_1000_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps
print(fps)
if cap.isOpened() == False:
    print("Cannot open video file.")

119.88011988011988


In [397]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)


In [398]:
#defining size for output videos
output = cv2.VideoWriter('/Users/danielrosenthal/Documents/EGR_314/colorblock_output2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1920,1080))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [399]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [400]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])



In [401]:
#defining rgb range to detect orange for now
orange_lower_bound = np.array([0,70,188])
orange_upper_bound = np.array([30,255,255])

blue_lower_bound = np.array([100,80,80])
blue_upper_bound = np.array([170,255,255])

yellow_lower_bound = np.array([30, 100, 100])
yellow_upper_bound = np.array([80, 255, 255])

In [402]:
#conversion factor, likely incorrect bc camera and setup need to be calibrated
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess




In [403]:
#initializing
conversion_rate = 0
rect_coordinates = 0
new_data = None
counter = 0
prev_frame = None
curr_frame = None
threshold = 500
x = 0
y = 0
centers_and_labels = {}
prev_centers_and_labels = {}
distance_threshold = 50
prev_blue_centers = []
prev_orange_centers = []
prev_x, prev_y = None, None
prev_time = None
time_interval = 1/fps
avg_rpm_blue = 0
avg_rpm_orange = 0

In [404]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    
    height, width, _ = frame.shape
    
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    #trying rgb threshold
    mask = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
    mask_blue = cv2.inRange(mask,blue_lower_bound,blue_upper_bound)
    mask_yellow = cv2.inRange(mask,yellow_lower_bound, yellow_upper_bound)
    mask_orange = cv2.inRange(mask,orange_lower_bound,orange_upper_bound)
    
    yellow_contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    blue_contours, _ = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    orange_contours, _ = cv2.findContours(mask_orange,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    blue_contours = [cnt for cnt in blue_contours if cv2.contourArea(cnt) > threshold]
    orange_contours = [cnt for cnt in orange_contours if cv2.contourArea(cnt) > threshold]
    
    # mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    # rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    contour_labels = []

    longest_width = 0
    for cnt in yellow_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (255,0,255), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if w > longest_width:
                longest_width = w
            #print(longest_width)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255,0,255),3)
                
    
            
    if longest_width > 0:
        conversion_rate = disc_length / longest_width
    

    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = (dx / dt) 
        vy = (dy / dt)
        
        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate
        vy_mph = vy * conversion_rate
        velocity = np.linalg.norm([vx_mph, vy_mph])

        # Display the velocity values on the output video
        cv2.putText(frame, 'X Velocity: {:.2f} mph'.format(vx_mph), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'Y Velocity: {:.2f} mph'.format(vy_mph), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'True Velocity: {:.2f} mph'.format(velocity), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})


# everything below here is trying to calculate spin rate

    # Find the center of each contour
    blue_centers = []
    for cnt in blue_contours:
        M = cv2.moments(cnt)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        blue_centers.append([cX, cY])

    orange_centers = []
    for cnt in orange_contours:
        M = cv2.moments(cnt)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        orange_centers.append([cX, cY])
   
    # Calculate the spin rate of the disc
    if prev_blue_centers and prev_orange_centers:
        # Calculate the change in the positions of the centers
        delta_blue_centers = np.array(blue_centers) - np.array(prev_blue_centers)
        delta_orange_centers = np.array(orange_centers) - np.array(prev_orange_centers)
        
        # Calculate the angular velocity of the disc in radians per second
        spin_rate_blue = np.arctan2(delta_blue_centers[:,1], delta_blue_centers[:,0]) / time_interval
        spin_rate_orange = np.arctan2(delta_orange_centers[:,1], delta_orange_centers[:,0]) / time_interval
        
        # Calculate the average spin rate
        avg_spin_rate_blue = np.mean(spin_rate_blue)
        avg_spin_rate_orange = np.mean(spin_rate_orange)

        #convert to rpm
        avg_rpm_blue = avg_spin_rate_blue * 30 / np.pi
        avg_rpm_orange = avg_spin_rate_orange * 30 / np.pi

        #convert to rps
        avg_rps_blue = avg_spin_rate_blue / (2*np.pi)
        avg_rps_orange = avg_spin_rate_orange / (2*np.pi)
        
        print("Blue spin rate:", avg_rps_blue, "rps")
        # print("Orange spin rate:", avg_rps_orange, "rps")
    
    # Update the previous centers of the blue and orange quadrants
    prev_blue_centers = blue_centers
    prev_orange_centers = orange_centers
    

    for i in range(len(blue_contours)):
        cv2.drawContours(frame, [blue_contours[i]], 0, (0, 0, 255), 2)
        cv2.putText(frame, 'Blue RPM: {:.2f} rpm'.format(avg_rpm_blue), (800, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    for i in range(len(orange_contours)):
        x, y, w, h = cv2.boundingRect(orange_contours[i])
        # cv2.putText(frame, str(centers_and_labels[i][1]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        # cv2.line(frame, tuple(centers_and_labels[i][1]), tuple(prev_centers_and_labels[i][0]), (255, 0, 0), 2)
        cv2.drawContours(frame, [orange_contours[i]], 0, (0, 0, 255), 2)
        cv2.putText(frame, 'Orange RPM: {:.2f} rpm'.format(avg_rpm_orange), (800, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
   
    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time

    cv2.imshow("frame", frame)
    # cv2.imshow("frame keypoints", frame_keypoints)
    #cv2.imshow("roi", roi)
    #cv2.imshow("Mask", mask)
    # cv2.imshow("mask yellow",mask_yellow)
    
    # output.write(frame)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

Blue spin rate: -3.119050934735971 rps
Blue spin rate: -3.7998611810740566 rps
Blue spin rate: -5.385334337589926 rps
Blue spin rate: -6.014143168897497 rps
Blue spin rate: -6.6917186786229825 rps
Blue spin rate: -5.677332991045561 rps
Blue spin rate: -4.347401672925941 rps
Blue spin rate: -1.9571736951858596 rps
Blue spin rate: 0.5373090019584356 rps
Blue spin rate: 2.027481902828896 rps
Blue spin rate: 2.6758943425024286 rps
Blue spin rate: 2.4746613290617736 rps
Blue spin rate: 1.1666772043904288 rps
Blue spin rate: 0.0 rps
Blue spin rate: -1.679137885171799 rps
Blue spin rate: -3.4657713328305215 rps
Blue spin rate: -4.825095768859289 rps
Blue spin rate: -5.926070237739054 rps
Blue spin rate: -6.845169903291197 rps
Blue spin rate: -6.4100985257015415 rps
Blue spin rate: -5.309822144829455 rps
Blue spin rate: -2.8061834903365543 rps


ValueError: operands could not be broadcast together with shapes (0,) (1,2) 

In [ ]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

   x_velocity  y_velocity
0    0.000000    0.000000
0    0.000000  203.173699
0   13.802114   -6.210951
0   35.627050   -3.995557
0   25.516785   -4.819837
0   23.658484   -8.603085
0   24.382174  -12.468157
0   22.568404  -20.419033
0   22.233402  -28.504362
0   15.662784  -27.544896
0    9.374932  -28.124795
0   -1.152070  -28.225708
0   -2.214641  -18.270788
0   -3.282859   -9.848577
0   -2.819397   -6.766553
0    0.556679   -5.010112
